# 🚀 Zip File Creator and Downloader with ipywidgets

This Jupyter Notebook provides an interactive way to create a zip archive of a selected folder and then download the resulting zip file. It uses `ipywidgets` for the user interface and standard Python libraries for file handling and zipping.

## How it Works

The notebook is built around a few key components:

1.  **Folder Selection:**

    *   **Text Input:** A `Text` widget allows you to manually enter the path to the folder you want to zip.
    *   **`ipyfilechooser` : A widget for file and folder choosing. 
    *   **Error Handling:** The code checks if the entered path is a valid directory.  If not, it displays an error message in a dedicated `Output` widget.

2.  **Zip File Naming:**

    *   **Text Input:** Another `Text` widget lets you specify the name of the zip file (without the `.zip` extension, which is added automatically).
    *   **Input Validation:** The code ensures that you've entered a name.  Leading/trailing whitespace is removed.

3.  **Zipping and Downloading:**

    *   **"Zip and Download" Button:**  Clicking this button triggers the main zipping and download process.
    *   **Temporary Directory:** A temporary directory is created *within the notebook's working directory* to store the zip file *before* downloading.  This is crucial to prevent the zip file from being included in the archive itself!  This temporary directory is cleaned up afterward.
    *   **`zipfile` Module:** The core zipping is handled by Python's built-in `zipfile` module:
        *   `zipfile.ZipFile(..., 'w', zipfile.ZIP_DEFLATED)`: Creates a new zip file in write mode (`'w'`) with DEFLATE compression (to reduce file size).
        *   `os.walk()`:  Recursively walks through the selected folder, getting all files and subfolders.
        *   `os.path.relpath()`:  Calculates the *relative* path of each file within the folder.  This is essential for maintaining the correct directory structure within the zip archive.
        *   `zipf.write(file_path, relative_path)`: Adds each file to the zip archive, using the calculated relative path to preserve the folder structure.
    *   **`IPython.display.FileLink`:**  After the zip file is created, `FileLink` generates a clickable link in the notebook's output area. Clicking this link initiates the download.
    *   **Error Handling:** The entire process is wrapped in a `try...except...finally` block.  This ensures that:
        *   Any errors during the zipping process (e.g., file access problems) are caught.
        *   An error message is displayed to the user.
        *   The temporary directory is *always* deleted, even if an error occurred. This prevents clutter.
    *   **Output Widget:**  Messages (success or error) are displayed in a dedicated `Output` widget (`output_area`). This provides feedback to the user.  The output area is cleared before each operation.

## How to Use

1.  **Install Dependencies:**  Make sure you have `ipywidgets` and `jupyter` installed (`pip install ipywidgets jupyter`).
2.  **Install `ipyfilechooser` (Recommended):**  `pip install ipyfilechooser`.  Then, follow the instructions in the code comments to replace the "Browse" button workaround with the `ipyfilechooser` dialog.
3.  **Run the Notebook:**  Open the `.ipynb` file in Jupyter Notebook or JupyterLab.
4.  **Select a Folder:**  Either type the folder path into the "Folder" text box or use the "Browse" button (or the `ipyfilechooser` dialog if you've installed it).
5.  **Enter a Zip File Name:**  Type the desired name for your zip file in the "Zip Name" box (don't include `.zip`).
6.  **Click "Zip and Download":**  This will create the zip file and display a download link.
7. **Download:** Click on the displayed link to download the zip file.

## Key Improvements Over Simpler Approaches

*   **Robust Error Handling:**  Handles various potential errors (invalid paths, file access issues) gracefully.
*   **Temporary Directory:** Prevents the zip file from being included in itself, a common problem.
*   **Relative Paths:**  Preserves the directory structure within the zip archive correctly.
*   **Compression:**  Uses DEFLATE compression to reduce the zip file size.
*   **Clean Download Link:** Provides a direct download link using `FileLink`.
*   **Organized UI:** Uses `HBox` and `VBox` for a better layout.
*   **Message Output:**  Displays informative messages to the user.
* **Clear Previous Output:** prevents old messages stacking up.
* **Comprehensive Explanations:** Detailed comments and this markdown cell make the code easier to understand.

This notebook provides a user-friendly and reliable way to create zip archives of folders directly within a Jupyter environment.  The use of `ipywidgets` makes it interactive, while the robust error handling and temporary directory usage ensure a smooth and correct zipping process.

**Community & Support:**

*   **GitHub:** [Ktiseos Nyx @ Github](https://github.com/Ktiseos-Nyx/HuggingFace_Backup) (for updates, bug reports, and contributions)
*   **Discord:**
    *   [Ktiseos Nyx AI/ML Discord](https://discord.gg/HhBSvM9gBY)
    *   [Earth & Dusk Media](https://discord.gg/5t2kYxt7An)

In [3]:
!pip install ipywidgets jupyter
!pip install ipyfilechooser

In [6]:
import ipywidgets as widgets
from IPython.display import display, FileLink
import zipfile
import os
import shutil
from ipywidgets import Layout
from ipyfilechooser import FileChooser

# --- 1. Folder Selection Widget (using ipyfilechooser) ---

folder_chooser = FileChooser(os.getcwd())
folder_chooser.title = "Select a Folder"

folder_selector = widgets.Text(
    value='',
    placeholder='Enter folder path or select using the chooser',
    description='Folder:',
    disabled=False,
    layout=Layout(width='80%')
)

def on_folder_selected(chooser):
    if chooser.selected:
        folder_selector.value = chooser.selected

folder_chooser.register_callback(on_folder_selected)

folder_selection_vbox = widgets.VBox([folder_chooser, folder_selector])

# --- 2. Zip File Name Widget ---

zip_name_input = widgets.Text(
    value='my_archive',
    placeholder='Enter zip file name (without .zip)',
    description='Zip Name:',
    disabled=False,
    layout=Layout(width='50%')
)

# --- 3. Zip and Download Button ---

zip_button = widgets.Button(description="Zip and Download")
output_area = widgets.Output()

def zip_and_download(b):
    with output_area:
        output_area.clear_output()
        folder_path = folder_selector.value
        zip_file_name = zip_name_input.value.strip()

        if not folder_path:
            print("Please select a folder.")
            return
        if not zip_file_name:
            print("Please enter a zip file name.")
            return
        if not os.path.isdir(folder_path):
            print("Invalid folder path. Please provide a valid directory")
            return

        zip_file_path = zip_file_name + '.zip'
        # Path in the *notebook's* directory (where the link will work)
        final_zip_path = os.path.join(os.getcwd(), zip_file_path)

        try:
            temp_dir = "temp_zip_dir"
            os.makedirs(temp_dir, exist_ok=True)
            temp_zip_path = os.path.join(temp_dir, zip_file_path)

            with zipfile.ZipFile(temp_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
                for root, _, files in os.walk(folder_path):
                    for file in files:
                        file_path = os.path.join(root, file)
                        relative_path = os.path.relpath(file_path, folder_path)
                        zipf.write(file_path, relative_path)

            # *** Copy the zip file to the notebook's directory ***
            shutil.copy2(temp_zip_path, final_zip_path)

            print(f"Successfully created {zip_file_path}")
            # *** Use the path in the notebook's directory for FileLink ***
            display(FileLink(final_zip_path))

        except Exception as e:
            print(f"Error creating zip file: {e}")
        finally:
            shutil.rmtree(temp_dir, ignore_errors=True)

zip_button.on_click(zip_and_download)

# --- 4. Display Widgets ---

display(widgets.VBox([folder_selection_vbox, zip_name_input, zip_button, output_area]))